In [5]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Union, List

## Constants

In [12]:
M = 1 # kg
x0 = 2.8 # m
v0 = 0 # m/s
dt = 0.1
alpha = 0

## Helper Functions

In [13]:
def V(x: Union[np.ndarray, float]) -> Union[np.ndarray, float]:
    """
    Custom Potential Energy dependent on position
    """
    return -np.exp(-x**2)-1.2*np.exp(-(x-2)**2)  # J

In [14]:
def derivative(func, x0: float, n: int = 1) -> float:
    """
    Computes derivative of a function f where x=x0
    """
    dx = 1e-6
    if n == 1:
        # computes first derivative
        dy = func(x0+dx) - func(x0)
        return dy/dx
    if n > 1:
        # reduces higher derivatives to lower ones
        dy = derivative(func, x0 + dx, n-1) - derivative(func, x0, n-1)
        return dy / dx

In [15]:
def get_next_euler(x: float, v: float, dt: float) -> float:
    new_x = x + v * dt
    new_y = v - 1 / M * derivative(V, x) * dt - alpha * v * dt
    return new_x, new_y

In [16]:
def calculate_euler(t_range: int):
    t = np.linspace(0, t_range, t_range / dt)
    x = t.copy()
    v = t.copy()
    for i in range()
    